![book header](pictures/header.png)

In [1]:
# Import necessary libraries
import time
import numpy as np
import matplotlib.pyplot as plt

# Uncomment one of the following lines depending on your setup

# If you are using the real car, uncomment the next line and comment the simulator line
# from serial import Serial

# If you are using the simulator, uncomment the next line and comment the real car line
from KITT_Simulator.serial_simulator import Serial

# Note: After changing the import statement, you need to restart the kernel for changes to take effect.

# Chapter 3:
# Module 1 - Connecting with and Controlling KITT

In this module, we will start working with the car simulator and the car. We will provide an overview and a brief description of the vehicle, and implement some basic control scripts.

## Brief Overview

KITT is a vehicle that you can control remotely, similar to a typical RC car. While traditional RC cars usually come with a joystick for control, this project has adapted the car so that you can use your computer instead. You can program your computer and use your keyboard to make KITT move forward, backward, or turn.

However, the goal of the project is to automate KITT, making it drive autonomously. As your computer will do the calculations, you will need to connect the car wirelessly to your computer. Over the wireless link, you can send commands to KITT, such as controlling its direction or speed. You can also request sensor data, which is a key difference from conventional RC cars. So this setup is a **two-way** communication link.

In this module, you will learn how to set up this wireless connection and start controlling KITT using basic keyboard commands. We will use object-oriented programming. We will also do this in such a way that, by changing one line, you can swap the real car for a simulated (model) car, allowing you to test your scripts in a simulation before going into the field.

### KITT Basics

KITT is built on a modified toy car model, the Traxxas E-MAXX (see picture below). In addition to its original components, KITT is equipped with two ultrasonic sensors, an LCD display, and the necessary electronics to transmit data via a Bluetooth connection. The LCD on the car provides real-time information, such as the readings from the distance measurement sensors and the battery voltage level.

In the Files>Datasheets>KITT datasheet.pdf, you can find a data sheet that gives a more detailed overview of the various components, such as the motor controller, LCD status indicator, ultrasonic sensors, and, most importantly, the communication module.

**Note:** As a heads-up for the future, when you start gathering data from KITT via the wireless link, you might notice that it doesn't always match what’s shown on the car's LCD. The data received directly from the car is more accurate, as the LCD might update at a slower rate. Therefore, while the LCD can serve as an initial indicator of any issues, it's best to rely on the direct data from the car for precise information.

<img src="pictures/traxxas_e-maxx.png" alt="Traxass E-MAXX along with it's dimensions" width="600" />

#### Hardware

KITT is equipped with a custom-made circuit board that brings together various components needed for its operation. At the heart of this board is a microcontroller, which acts as the main control unit.

This board also includes everything necessary for wireless communication, like the Bluetooth module, as well as connectors for other parts of the car. Additionally, there's an amplifier on the board that's used for sending out audio signals that help with localization.

The circuit board is powered by rechargeable batteries. These batteries supply the necessary voltage through a component that adjusts the voltage to the required levels.

Here are the key components included on the board:

- **Microcontroller:** NXP LPC4357 chip with an ARM CORTEX-M4/M0 core
- **Bluetooth module**
- **Amplifier for the audio beacon**
- **Buck converter** for voltage regulation
- **Voltage regulator:** LM1117-3.3 for generating 3.3V DC

<img src="pictures/communication_overview.png" alt="Schematic of communication with KITT and PC." width="600" />

KITT has a Bluetooth module for communication with your PC. It consists of the following elements:

- On car: Roving Networks RN-41 I/RM, onboard Bluetooth module with UART control.
- On PC: LM Technologies LM506, USB Bluetooth V4.0 dongle with Broadcom BCM20702 chipset, or the internal Bluetooth module of your laptop.

**Tip:** If you want to use one of the lab computers or have trouble connecting to KITT from your PC (check if all drivers are installed), you could use the USB Bluetooth dongle.

KITT has four LEDs on the front, see the figure below. Their meaning is as follows:

<img src="pictures/figurejoin.png" alt="KITT control board and front LEDs" width="600" />

- **Red (twice):** *5V* and *20V* supply voltages are present.
- **Green (blinking):** Bluetooth is searching for a connection; **(steady):** Bluetooth connected.
- **Yellow (blinking):** Bluetooth data transfer.

The button next to the LEDs is a reset button for the MCU. This reset button allows the user to reset KITT's actions if it freezes or is unresponsive.

#### Simulated Model

The KITT test fields have limited availability. To facilitate software development and testing, you can use a KITT software model. It is in the package `KITT_Simulator`. If you use the `Serial` command from this package and connect to KITT (see below), then instead of connecting to the real car, you connect to the software model. You can send commands and read its status using the same commands as for the real KITT. Thus, you can first test all your algorithms on the simulated model and later on the real KITT.

Of course, this only works if the model is sufficiently close. At some point, you'll have to adjust the parameters in the your code to match the reality of your car (calibration).

To toggle between running the simulated model or the real car, look at the top block of this notebook: by commenting or uncommenting the appropriate import line, you can switch between the real car and the simulator. You will need to restart the kernel after changing this command.

--- 

## Communicating with KITT

In this section, we cover the essential concepts required to effectively communicate with the KITT system. In the following section, we will apply these principles in practice, implementing them in the simulator and the car.

#### pySerial

`pySerial` is a Python module that provides a simple and efficient way to communicate with serial ports. It allows Python programs to access and manipulate the serial ports on a computer, allowing them to communicate with other devices connected to those ports. With `pySerial`, you can easily send and receive data to/from these devices. `pySerial` is cross-platform and works on Windows, macOS, and Linux operating systems.

- `Serial(port, baudrate)` - This command initializes a serial connection. The `port` argument specifies the serial port to use (e.g., "COM1" on Windows or "/dev/rfcomm0" on Linux), and the `baudrate` argument specifies the data rate in bits per second.
- `serial.write(data)` - This command sends data over the serial connection. The `data` argument is a bytes object that contains the data to be sent.
- `serial.read(size)` - This command reads a specified number of bytes from the serial connection. It blocks until the specified number of bytes are received.
- `serial.read_until(bytes)` - This command reads bytes from the serial connection until the specified byte sequence is found. It blocks until then.
- `serial.flush()` - This command is used to flush the input and output buffers of the serial connection.
- `serial.close()` - This command is used to close the serial connection.

More detailed and updated information regarding the module can also be found through this link: [pySerial Documentation](https://pyserial.readthedocs.io/en/latest/pyserial.html).

#### Simulated Serial Port

The KITT simulator (car model) is also controlled by a serial port. This port uses the following functions:

- `Serial(port, baudrate)`
- `serial.write(data)`

These functions work identically to the `pySerial` functions, except that they communicate with the simulated KITT and not the real KITT.

#### Controlling KITT

After connecting to KITT, several commands are available to control it: driving commands, audio beacon commands, and status commands.

**Driving Commands:** For driving, there are two types of instructions:

- A direction command: `D`. Example: `D130`
- A motor speed command: `M`. Example: `M160`

These control the motors using Pulse Width Modulation (PWM). Both commands are neutral at a setting of 150. The direction commands range from 200 (hard left) to 100 (hard right), and the motor commands range from 135 (backward) to 165 (forward).

**Note:** There is a dead zone for the motor commands, so KITT will likely not start moving forward until the PWM is set to about 153. It is recommended to experiment with these values; they are also battery-dependent. You should also verify that 150 is the middle position for steering (sometimes there is a deviation of ±2), and test the maximum left and right positions.

All the commands are sent in binary over the serial link and end with a new line character.

```python
serial.write(b'code\n')
```

To set direction to 130 and motor speed to 160, it can be done as follows:

```python
serial.write(b'D130\n')
serial.write(b'M160\n')
```

**Note:** Once you set the motor speed commands, KITT will continue to act on this until you either transmit a new motor speed command or reset the MCU using the button on KITT, which will set both direction and motor speed commands to the neutral value of 150. To be safe, include suitable stop commands in your scripts.

**Simulator Example:** Let's see this in the simulator:

In [ ]:
# Open serial port
serial = Serial('/dev/ttyUSB0', 115200)

# Wait for one second to allow for computer processing delay
time.sleep(1)

# Set speed and direction
serial.write(b'M162\n')
serial.write(b'D100\n')
print("Motors are ON")

time.sleep(2)

# Set speed to zero
serial.write(b'M150\n')
print("Motors are OFF")

time.sleep(5) # Notice the car keeps moving (rolling out till standstill)

# Close the connection (important!)
serial.close()

In [ ]:
### Student Version ###

# Open the serial port with the correct port and baud rate
# Replace '/dev/ttyUSB0' with your actual serial port
serial = Serial('/path/to/serial/port', 115200)

# Allow time for any processing delays
time.sleep(1)

# TODO: Set a motor speed and direction
# Use serial.write() to send the motor speed command
# Example: serial.write(b'M<speed_value>\n')
# Use serial.write() to send the direction command to turn left
# Example: serial.write(b'D<direction_value>\n')
# Hint: To turn left, set the direction value greater than 150 (neutral position)
print("Motors are ON")

# Wait for some time to observe the car's movement
time.sleep(2)

# Stop the car by setting motor speed to neutral (150)
# Use serial.write() to send the stop command
serial.write(b'M150\n')
print("Motors are OFF")

# Wait to observe the car slowing down
time.sleep(5)  # Notice the car keeps moving due to inertia

# Close the serial connection (important!)
serial.close()

**Additional Information:**

*To make the car turn left instead of right, adjust the direction value in the `D` command accordingly. Remember that direction values range from 100 (hard right) to 200 (hard left), with 150 being neutral.*

Also, notice that the car keeps moving after the motors are off. This is because of the car's dynamics, which will be explored in a later module.

**Audio Beacon**

The car has an audio beacon which will be used to localize the car. The beacon parameters can be programmed using the following commands:

- `F`: Set transmission frequency (Timer 0)
- `B`: Set bit frequency (Timer 1)
- `R`: Set repetition count (Timer 3)
- `C`: Set 32-bit code
- `A`: Turn beacon on/off

You can set the beacon parameters using the following commands:

```python
serial.write(b'F1000\n')
serial.write(b'B5000\n')
serial.write(b'R2500\n') 
serial.write(b'C12345678\n')
serial.write(b'A1\n')  # Turn beacon on
serial.write(b'A0\n')  # Turn beacon off
```

<img src="pictures/Beacon.png" alt="Illustration of the transmitted signal" width="400" />

The repetition count formula is as follows:

$$\text{Repetition Count} = \frac{\text{Bit Frequency}}{\text{Repetition Frequency}}$$

For example, if the bit frequency is 5000 Hz, the repetition count should be 2500 to sound the beacon twice per second.

Later, you will use this beacon to determine where your car is on the field. Remember that all the numbers provided here serve only as examples. It is up to you to determine what carrier frequency, code word, etc., best fits the goal of succeeding at the final challenge.

Let's use these in the simulator. Unfortunately, the sound of the car cannot be heard in the simulation. A blue circle will appear on the car to indicate the when the beacon is on.

In [ ]:
serial = Serial('/dev/ttyUSB0', 115200)

# Carrier frequency set to 10000 Hz for example.
carrier_frequency = (10000).to_bytes(2, byteorder='big') # Convert number to bytes
serial.write(b'F' + carrier_frequency + b'\n') # `F` command sets carrier frequency

# Bit frequency set to 5000 Hz for example.
bit_frequency = (5000).to_bytes(2, byteorder='big')
serial.write(b'B' + bit_frequency + b'\n') # `B` command sets bit frequency

# Repetition count should be at least 32.
repetition_count = (2500).to_bytes(2, byteorder='big')
serial.write(b'R' + repetition_count + b'\n') # `R` command sets repetition count

# The 32 bits code pattern is transmitted bit-wise over the beacon. Hence, code must be specified in hexadecimal.
# The following code is a sequence of 16 `1` bits followed by 16 `0` bits (not at all a good code!)
# You will need to create your own code later
code = (0xFFFF0000).to_bytes(4, byteorder='big') # Convert hexadecimal number to bytes
serial.write(b'C' + code + b'\n') # `C` command sets code

time.sleep(1)
print("Beacon is ON")
serial.write(b'A1\n') # Turn Beacon ON
time.sleep(3)
print("Beacon is OFF")
serial.write(b'A0\n') # Turn Beacon OFF
time.sleep(1)

# Close the connection (important!)
serial.close()

In [ ]:
### Student Version ###

# Open the serial port with the correct port and baud rate
# Replace '/dev/ttyUSB0' with your actual serial port
serial = Serial('/path/to/serial/port', 115200)

# Set carrier frequency
# Define the desired carrier frequency in Hz (e.g., 10000)
carrier_frequency_value = 10000  # Replace with your value
# Convert the frequency to bytes (2 bytes, big endian)
carrier_frequency = carrier_frequency_value.to_bytes(2, byteorder='big')
# TODO: Send the 'F' command with the carrier frequency

# Set bit frequency
# Define the desired bit frequency in Hz (e.g., 5000)
bit_frequency_value = 5000  # Replace with your value
# Convert the bit frequency to bytes
bit_frequency = bit_frequency_value.to_bytes(2, byteorder='big')
# TODO: Send the 'B' command with the bit frequency

# Set repetition count
# Calculate the repetition count (should be at least 32)
# Repetition Count = Bit Frequency / Repetition Frequency
desired_repetition_frequency = 2  # Replace with your value
repetition_count_value = bit_frequency_value // desired_repetition_frequency
# Ensure repetition_count_value is at least 32
repetition_count_value = max(repetition_count_value, 32)
# Convert the repetition count to bytes
repetition_count = repetition_count_value.to_bytes(2, byteorder='big')
# TODO: Send the 'R' command with the repetition count

# Set code pattern
# Define a 32-bit code in hexadecimal (e.g., 0x12345678)
code_value = 0x12345678  # Replace with your code
# Convert the code to bytes (4 bytes, big endian)
code = code_value.to_bytes(4, byteorder='big')
# TODO: Send the 'C' command with the code

# TODO: Allow some time for the settings to take effect

# TODO: Turn the beacon ON by sending the 'A1' command
print("Beacon is ON")

# Wait while the beacon is on
time.sleep(3)

# TODO: Turn the beacon OFF by sending the 'A0' command
print("Beacon is OFF")

# Allow some time before closing
time.sleep(1)

# TODO: Very Important! Close the serial connection

**Additional Information:**

*Remember to choose appropriate values for the carrier frequency, bit frequency, and code pattern. The carrier frequency should not exceed 20 kHz, and the bit frequency should be smaller than the carrier frequency. The code pattern is a 32-bit value that will be transmitted bit-wise over the beacon.*

**Caution:** Be aware that the default code word for the beacon is `0x00000000`, which means KITT will not start making noise on its own when the beacon is turned on. You should specify a code as described above before you can hear the beacon make noise. Furthermore, an arbitrary carrier frequency, bit frequency, and repetition count can be used, but not all combinations make sense. (The carrier frequency should not be above 20 kHz; the bit frequency should be smaller than the carrier frequency.)

**Status Command**

Lastly, the status command, which requests the status and all the data from KITT, is `S`. The status string reports the current drive commands, the ultrasonic sensor distance (cm), the battery voltage (mV), the audio beacon status (on/off), and control parameters (code word, carrier frequency, bit frequency, repetition count). A sensor distance of 999 means overload (i.e., out of range).

**Additional Information:**

You can request information from KITT by sending different commands:

- **For all information**: Use the command:
  ```python
  serial.write(b'S\n')
  ```
  This returns all the information available from KITT.

- **For distance measurements only**: Use the command:
  ```python
  serial.write(b'Sd\n')
  ```
  This returns only the left and right distance sensor values.

- **For battery voltage**: Use the command:
  ```python
  serial.write(b'Sv\n')
  ```
  This command returns just the battery voltage level.

- **For version information**: Use the command:
  ```python
  serial.write(b'V\n')
  ```
  This sends back the version details of KITT’s firmware.

After sending each command, you have to read out what KITT sends back. The response is a string of bytes that you should best decode before interpreting it.:

```python
status = serial.read_until(b'\n')
status = status.decode('utf-8')
print(status)
```

In [ ]:
serial = Serial('/dev/ttyUSB0', 115200)

serial.write(b'S\n')

status = serial.read_until(b'\x04')
status = status.decode('utf-8')

print(f"Car status is:\n\n{status}")

# Close the connection (important!)
serial.close()

In [ ]:
### Student Version ###

# TODO: Open the serial port with the correct port and baud rate

# TODO: Send the status command 'S\n'

# TODO: Read the response until the End-of-Transmission character (EOT): '\x04'

# TODO: Decode the status information received from the car

# Print the status information received from the car
print(f"Car status is:\n\n{status}")

# TODO: Close the serial connection (important!)

## Implementing a Basic KITT Control Script

Now you can start writing a Python program to control the simulator or the car connected over Bluetooth. This program will allow you to control the car using the keyboard keys `w`, `a`, `s`, and `d`. The `e` and `r` keys will start and stop the audio beacon, respectively. Additionally, include a key to stop communication with KITT.

We begin by defining the `KITT` class and some essential methods for communication. Classes define the structure and capabilities of objects, allowing you to create multiple instances with the same properties and methods. Next, you will complete the `keyboard_control` function. We'll use a simple input loop to read keyboard commands. You will finally call the function and test your code first on the simulator and then on the real car.

### Explanation of the Code

**KITT Class:**

- **`__init__` Method:**
  - Initializes the serial communication with the specified port and baud rate.
  - Sets up the beacon parameters using the existing `send_command` method.
- **`send_command` Method:**
  - Sends a given command to KITT over the established serial connection.
- **`set_speed` and `set_angle` Methods:**
  - Adjust KITT’s speed and steering angle, respectively.
- **`stop` Method:**
  - Brings KITT to a halt by setting both speed and angle to their neutral states.
- **`start_beacon` and `stop_beacon` Methods:**
  - Control the audio beacon by sending the appropriate commands.
- **`close` Method:**
  - Ensures the proper closure of the serial communication when you're done using KITT.

**`keyboard_control` Function:**

- Designed as a continuous loop that reads keyboard commands from the user input.
- Interprets the input command and adjusts KITT’s speed and steering angle or toggles the beacon accordingly.
- **Key Mappings:**
  - `w`: Accelerate KITT forward.
  - `s`: Decelerate or move KITT backward.
  - `d`: Turn KITT right.
  - `a`: Turn KITT left.
  - `e`: Start the audio beacon.
  - `r`: Stop the audio beacon.
  - `q`: Stop communication with KITT and exit the program.
- Ensures that speed and steering values stay within the valid ranges.
- Handles exceptions and ensures resources are properly closed when exiting.

**Test Code:**

- The `if __name__ == "__main__"` block contains the code that will execute when you run your script.
- An instance of `KITT` is created with the appropriate serial port.
- The `keyboard_control` function is called with the `kitt` instance to start controlling the car.

**TODO:**

- **Modify the `__init__` Method:**
  - When the communication with KITT is started, initialize the beacon with the correct set of parameters using the existing `serial` port and `send_command`.
- **Add `start_beacon` and `stop_beacon` Methods:**
  - These methods will turn the beacon on or off. Since you set the beacon parameters during `__init__`, there is no need to resend them every time you turn the beacon on.
- **Implemented a Simple Input Loop:**
  - Using `input()` to read commands avoids issues with keybindings in VSCode or Jupyter notebooks.
  - This method is more compatible and less likely to cause kernel crashes.
- **Handled Exceptions and Resource Cleanup:**
  - Added a `try...except...finally` block to handle any exceptions that might occur.
  - Ensured that the car is stopped and the serial connection is closed when exiting the program.

In [ ]:
class KITT:
    def __init__(self, port, baudrate=115200):
        self.serial = Serial(port, baudrate, rtscts=True)
        # Initialize beacon parameters
        self.carrier_frequency = 10000
        self.bit_frequency = 5000
        self.repetition_count = 32
        self.code = 0xFFFF0000

        self.set_beacon()
    
    def set_beacon(self):
        # Set carrier frequency
        carrier_frequency = self.carrier_frequency.to_bytes(2, byteorder='big')
        self.serial.write(b'F' + carrier_frequency + b'\n')

        # Set bit frequency
        bit_frequency = self.bit_frequency.to_bytes(2, byteorder='big')
        self.serial.write(b'B' + bit_frequency + b'\n')

        # Set repetition count
        repetition_count = self.repetition_count.to_bytes(2, byteorder='big')
        self.serial.write(b'R' + repetition_count + b'\n')

        # Set code pattern
        code = self.code.to_bytes(4, byteorder='big')
        self.serial.write(b'C' + code + b'\n')

    def send_command(self, command):
        self.serial.write(command.encode())

    def set_speed(self, speed):
        self.send_command(f'M{speed}\n')

    def set_angle(self, angle):
        self.send_command(f'D{angle}\n')

    def stop(self):
        self.set_speed(150)
        self.set_angle(150)

    def start_beacon(self):
        # Send commands to start the beacon
        self.send_command('A1\n')

    def stop_beacon(self):
        # Send commands to stop the beacon
        self.send_command('A0\n')

    def close(self):
        self.serial.close()

def wasd_control(kitt):
    print("Control the car with W (forward), S (backward), A (left), D (right), E (start beacon), R (stop beacon), and Q (to exit).")
    car_speed = 150
    car_steering = 150

    try:
        while True:
            command = input("Enter command: ").lower()

            if command == 'w':
                car_speed += 5
            elif command == 's':
                car_speed -= 5
            elif command == 'a':
                car_steering += 15
            elif command == 'd':
                car_steering -= 15
            elif command == 'e':
                kitt.start_beacon()
                print("Beacon started.")
                continue
            elif command == 'r':
                kitt.stop_beacon()
                print("Beacon stopped.")
                continue
            elif command == 'q':
                kitt.stop()
                print("Exiting control.")
                break
            else:
                print("Invalid command.")
                continue

            # Ensure speed and steering values are within valid ranges
            car_speed = min(max(car_speed, 135), 165)
            car_steering = min(max(car_steering, 100), 200)

            kitt.set_speed(car_speed)
            kitt.set_angle(car_steering)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        kitt.stop()
        kitt.close()

# Example usage
if __name__ == "__main__":
    kitt = KITT('/dev/ttyUSB0')  # Replace with the actual port
    wasd_control(kitt)

In [9]:
### Student Version ###

class KITT:
    def __init__(self, port, baudrate=115200):
        # Initialize the serial connection
        # self.serial = Serial(port, baudrate, rtscts=True)
        
        # Initialize beacon parameters here using send_command
        # Set carrier frequency, bit frequency, repetition count, and code pattern
        pass  # Replace with actual initialization code

    def send_command(self, command):
        # Send the command string over the serial connection
        self.serial.write(command.encode())
        pass

    def set_speed(self, speed):
        # Send the motor speed command using send_command
        # Use the format 'M<speed>\n'
        pass

    def set_angle(self, angle):
        # Send the steering angle command using send_command
        # Use the format 'D<angle>\n'
        pass

    def stop(self):
        # Stop the car by setting speed and angle to neutral (150)
        # Call set_speed and set_angle with 150
        pass

    def start_beacon(self):
        # Send commands to start the beacon
        # Use the command 'A1\n'
        pass

    def stop_beacon(self):
        # Send commands to stop the beacon
        # Use the command 'A0\n'
        pass

    def close(self):
        # Close the serial connection
        # self.serial.close()
        pass

In [10]:
### Student Version ###

def wasd_control(kitt):
    print("Control the car with W (forward), S (backward), A (left), D (right), E (start beacon), R (stop beacon), and Q (to exit).")

    car_speed = 150  # Neutral speed
    car_steering = 150  # Neutral steering

    try:
        while True:
            # Read input from the user
            command = input("Enter command: ").lower()

            # TODO: depending on the command, adjust the car speed, steering angle, or start/stop the beacon variable

            # Ensure speed and steering values are within valid ranges
            car_speed = min(max(car_speed, 135), 165)
            car_steering = min(max(car_steering, 100), 200)

            # Send the speed and angle to KITT
            kitt.set_speed(car_speed)
            kitt.set_angle(car_steering)

    except Exception as e:
        # Handle any exceptions
        print(f"An error occurred: {e}")
    finally:
        # Ensure the car is stopped and serial connection is closed
        kitt.stop()
        kitt.close()

In [ ]:
### Student Version ###

if __name__ == "__main__":
    # Create an instance of KITT with the correct serial port
    # Replace '/dev/ttyUSB0' with your actual serial port
    kitt = KITT('/path/to/serial/port')  # Replace with the actual port

    # Start the control function
    wasd_control(kitt)

#### To Test the Code with the Real Car

Now you can check and see if you can also control the real car with the code you wrote. For that, you need to restart the kernel and uncomment the appropriate `Serial` import at the top of the notebook. No code changes are required.

#### Connecting to the Real KITT

Although the code does not need to be changed to work on the real car, theory and practice may differ. If you encounter any difficulties with connecting to KITT, read on.

To start with, connect the car to your computer over Bluetooth. Make sure you know the name of the Bluetooth port that the car is connected to. To access the link in Python, use:

```python
serial_port = Serial(comport, 115200, rtscts=True)
```

`serial_port` is an instance of `pySerial`, and this object can be used to manipulate KITT. The variable `comport` is the path to the port assigned to the Bluetooth connection with KITT. Which port specifically KITT is connected to can be found in your communication settings in Windows, etc.

For example, if your transmission connection to KITT takes place over port 10, then you should use:
```python
comport = 'COM10'
```

Connecting to a serial port over Bluetooth can be tricky. Windows users, refer to Appendix B. Linux users can use `rfcomm` commands to bind the MAC address to a serial port; there is also a section about it on the [Arch Linux wiki](https://wiki.archlinux.org/title/Bluetooth#Bluetooth_serial). The MAC address of KITT always ends with the code printed on its label. For example, if the label reads DA:84, the MAC will end with `xx:xx:xx:xx:DA:84`.

To find out which serial ports are available, you can use the following Python code:

In [ ]:
import serial.tools.list_ports

ports = list(serial.tools.list_ports.comports())
for port in ports:
    print(port.device)

**Note:** Keep in mind that the Bluetooth connection can be disrupted by leaving ports open, quitting your work without closing the communication link, removing the Bluetooth dongle, or turning off the Bluetooth connection. Avoid disturbing the Bluetooth connection by ending the connection properly before doing any of the aforementioned things. If you disturb the Bluetooth link, you may need to reboot your computer to reset the operating system, which costs you valuable lab time.

The connection must also be closed once you are done using the port:
```python
serial_port.close()
```

#### Optional Extensions

If you would like to go above and beyond with the task, here are a few tips that can help with extending the functionality of the program:

- **Add Error Handling:** Currently, if there is an error with the Bluetooth connection or the serial communication, the program will simply crash. You could add some error handling to handle these cases more gracefully, such as printing an error message and exiting the program.
- **Add Speed Control:** Currently, the program only supports moving the car forward or backward and turning left or right. You could add support for controlling the speed of the car, such as by sending different commands to the car depending on how long the user holds down the forward or backward keys.
- **Add an Emergency Stop:** KITT doesn’t have a brake; the 150 speed setting lets KITT roll out to standstill, which might take a long distance. Add an emergency brake by letting KITT drive backwards for a short period of time. (You should first detect if the previous speed setting caused KITT to move forward. You need to define a state variable to memorize the speed setting.)

#### Mid-term Assessment and Report

After you finish this assignment, and ultimately in week 4, showcase the functionality of your script to your assigned TA. After you pass this assessment, you are ready to document your results in your midterm report.

For this module, you should include a chapter that covers the approach, implementation, testing, and results of the basic controller.

Remember to document your code, using comments to define input/output variables of functions and to explain the logic and any modifications made. Your completed script will be crucial for the upcoming challenges, contributing to the overall autonomous driving system.

After you have completed this module and have tested all the components thoroughly, start on the second part of the communication script outlined in Module 2.

---
# **FAQ**



**What is a serial communication and why is it used?**

Serial communication is a way of transmitting data one bit at a time over a single communication line. It's commonly used to connect devices like computers and microcontrollers because it simplifies wiring and is efficient for long-distance communication. In this method, data is sent in a sequence, or "serially," from the sender to the receiver, typically using standards like USB, RS-232, or UART.




**What is the COM port?**

In a Bluetooth connection, a COM port (short for "communication port") is a virtual port that allows your computer to communicate with Bluetooth devices as if they were connected via a traditional serial port (like those used for older wired connections). When you pair a Bluetooth device with your computer, the operating system assigns it a virtual COM port, which acts as an interface for data exchange between your computer and the Bluetooth device. This makes it easier to send and receive data over Bluetooth using standard serial communication methods.


**What is a buck converter and why is it used?**

A buck converter is a type of DC-DC converter that steps down voltage from a higher level to a lower level while maintaining efficient power transfer. It works by rapidly switching a transistor on and off to control the energy transferred to an output capacitor, which smooths out the voltage to a lower, steady level.

Buck converters are used because they are highly efficient, meaning they waste very little power as heat. This efficiency is particularly important in battery-powered devices, like KITT, where it's crucial to conserve energy and maintain a stable voltage for sensitive electronics. For instance, if the battery supplies 12V but the circuit requires 5V, a buck converter efficiently reduces the voltage to the needed level without generating much heat or wasting energy.




**I have paired the car with my PC, however the LED is still blinking; is the car connected?**

Pairing your Bluetooth module and assigning a COM port is different from running the Python script. If you only pair the device, you'll notice that the LED continues to blink after some time. However, when you run the Python script and the virtual COM port is properly allocated, the LED will stop blinking. Simply pairing the device isn't enough to stop the LED from blinking—you need to run the Python script as instructed to see the LED become stable which means there is an actual connection.

**What is pulse width modulation?**

Pulse Width Modulation (PWM) is a technique used to control the amount of power delivered to an electronic device by varying the width of the pulses in a signal. It works by switching a signal on and off rapidly at a constant frequency, and by adjusting the ratio of the "on" time to the "off" time, you can control the average power delivered to a load.

For example, if a signal is "on" for 50% of the time and "off" for the other 50%, the output will deliver 50% of the maximum power. This method is commonly used in applications like controlling the speed of motors, dimming LED lights, and generating analog signals from digital sources because it is efficient and provides precise control over power.


**I've implemented the code, and everything functions correctly in the GUI and runs without errors, but the car isn't moving. What should I do?**

Keep in mind that there's an additional switch that controls the power to the motors. If this switch is turned off, the motors won't receive any power from the battery, even if everything else is set up correctly; the car won't move. Make sure this switch is turned on.